![Cradlepoint](logo.svg)

---

# **PART 3** - Application Development

- #### Flow Control
- #### State Tracking
- #### Time Tracking
- #### NCM Config Sync
- #### Custom Alert Integration

---

## Import the **csclient** and **json** modules and instantiate the **CSClient()** class.

> At this point you should be able to do this on your own!

#### Good luck!

## [Flow Control](https://docs.python.org/3/tutorial/controlflow.html)

#### You can create apps that do something once and are done.  But it is much more common that you want an app to *constantly* do that one thing, or a bunch of things!  

#### Or maybe you want it to do something *only when a specific condition is met*...  Or based on a *time interval*...  This requires constantly monitoring that condition or time.

#### These are very common flows in applications and are easily handled with the use of **loops.**

### **Loops**

#### The simplest and most common approach to loops is the `while True:` loop.  This loops repeats always and forever (True is always True), unless you `break` out of it.  Many apps use this or a similar approach to always keep running.

#### Some things to consider when utilizing loops:
- Without accurate "if" conditions, you could DO something every time, which can easily spin out of control.
- Code runs as fast as possible, so if you don't slow your loop down (time.sleep()), it can consume the CPU.
- State tracking: tracking if something has changed since the last iteration (loop).
- Time tracking: tracking time deltas since the last time an action was performed.

#### We will address all of these points in this lab!

### **If statements**

#### Use **if** statements to take action when conditions are met.

#### Example:

First `.get()` the current WAN device UID from `status/wan/primary_device`.  Then use an `if` statement to verify primary_device exists (avoid exception) and then determine if it is a modem, then print something indicating so.

```python
primary_device = cp.get('status/wan/primary_device')
if primary_device.startswith('mdm'):  # Modem device
    print(f'primary_device: {primary_device} is a modem!')
else:
    print(f'primary_device: {primary_device} is NOT a modem!')
```

### **time.sleep()**

#### As a best practice, `import time` and add a `time.sleep(1)` in your loops to pause for 1 second and avoid consuming too much CPU:

### Conditional Loop Exercise

#### Tying these concepts together, create a `while True:` loop that gets a value, uses `if` to determine action, and does a `time.sleep(1)` at the end before repeating:

---
```python
import time

# Loop:
while True:
    
    # Get value:
    primary_device = cp.get('status/wan/primary_device')
    
    # If statements determine action:
    if primary_device.startswith('mdm'):  # Modem device
        print(f'primary_device: {primary_device} is a modem!')
    else:  # Not a modem device:
        print(f'primary_device: {primary_device} is NOT a modem!')
    time.sleep(1)
```
---

#### Your turn (There is a stop button on the toolbar at top):

> Extra Credit: Avoid NoneType exception from .startswith() if primary_device = None

## State Tracking

#### As you can see, this script writes a log every time it loops, regardless if the primary_device has changed or not.  In reality, you probably only want to know when it changes.  To do this you need to track the state and include it in your `if` statements in your loop.  

#### A simple approach is using a "previous" prefix on a tracking variable name to indicate it was the previous value you are comparing against (e.g. "previous_primary_device")

> The example below adds the tracking variable, and an `if` statement to action only when the primary_device has changed.

---
```python
# Tracking variable:
previous_primary_device = cp.get('status/wan/primary_device')

while True:
    # Current value:
    primary_device = cp.get('status/wan/primary_device')
    
    # Detect change:
    if primary_device != previous_primary_device:
        
        # Take action:
        if primary_device.startswith('mdm'):
            print(f'primary_device: {primary_device} is a modem!')
        else:
            print(f'primary_device: {primary_device} is NOT a modem!')
    time.sleep(1)
```
---

#### *Copy your script from the previous cell and modify it below to add the tracking.*

#### Your turn:

## Time Tracking

### Use the **time** module to track a timestamp and take action on a schedule.

#### Apply this concept to the previous script to only check the primary_device every 1 minute:

---
```python
# Time interval between checks (in seconds):
interval = 60

# Tracking variables:
previous_primary_device = cp.get('status/wan/primary_device')
previous_time = time.time()

while True:
    # If the difference between now and previous_time is greater than or equal to interval
    if time.time() - previous_time >= interval:

        # Current values:
        primary_device = cp.get('status/wan/primary_device')

        # Detect change:
        if primary_device != previous_primary_device:

            # Take action:
            if primary_device.startswith('mdm'):
                print(f'primary_device: {primary_device} is a modem!')
            else:
                print(f'primary_device: {primary_device} is NOT a modem!')
        time.sleep(1)
```
---

> Create larger intervals with simple math:  
> (seconds * minutes * hours * days)  
> 5 minutes: `interval = 60 * 5`  
> 24 hours: `interval = 60 * 60 * 24`  

#### *Copy your script from the previous cell and modify it below to add the time schedule.*

#### Your turn:

# NCM Config Sync

### Routers maintain a persistent connection to NCM, and bidirectionally sync configuration changes.

## NCM Config Change

#### There are multiple ways you can change a device configuration in NCM.  Any of the following will trigger NCM to rebuild and sync the config:

- Change Device Config (Including changing "Name", "Description", or "Asset ID" via the NCM Devices grid)
- Change Group Config that the device belongs to
- Move a Device in/out of Group
- Any of the above via NCM API

#### When NCM builds and syncs a config it goes through the following steps:

- **Patch** the **Device Config** into the **Group Config** resulting in the **Target Config**
- **Diff** between **Target Config** and the **Actual Config** resulting in the **Pending Config**
- Push the **Pending Config** to the device for application.

#### When the device receives the **Pending Config** it goes through the following steps:
- Start a 15 minute rollback timer
- Disconnect from NCM
- **Patch** the **Pending Config** to the local config - same as cp.patch()
- Reconnect to NCM and stop rollback timer.

> If rollback timer is reached (unable to reconnect to NCM), the configuration is **reverted** (undone), and the router will reconnected to NCM and set its "config_status" to "suspended" (Configuration Sync Suspended) preventing further config changes until told to resume updates.  This is a lifeline to prevent you from remotely disconnecting your devices via remote configuration changes.

## NCOS Config Change

#### Using any of the following to change a device configuration in NCOS (local device) will trigger NCOS to sync changes up to NCM.

- Web UI
- CLI
- REST API
- SDK

## Tie-Breaker

#### If changes are made to the device configuration both in NCM **AND** in NCOS *while the device is offline*, when it reconnects to NCM there will be a conflict.  NCM wins and the local changes are dropped.


## Leveraging NCM Config Sync for Application Integration

#### Because the device "name", "description", and "asset_id" fields are part of the device config and are bidirectionally synced, you can leverage these fields for transporting "messages" between SDK Applications and the NCM Devices Grid.

> #### For example, a speed test application runs a speed test and puts the results into the "asset_id" field, which syncs up to NCM so you can see the results of all of your devices at a glance.  Then you can just clear that "asset_id" field, which syncs back down to the device where an application can recognize that and trigger a new test to be run, placing the new results into the "asset_id" field to be synced up to NCM.


## Custom Alert Integration

### NCM supports sending [**alerts**](https://customer.cradlepoint.com/s/article/Alerting-and-Reporting-with-NCM) via [**Webhooks**](https://customer.cradlepoint.com/s/article/Using-NetCloud-Webhooks-to-Receive-Push-Data)

#### Alerts that are generated by a router (including custom alerts) are considered **router alerts** and include the following router fields in the body of the webhook message:

- "router_id"
- "name"
- "description"
- "mac"
- "serial_number"
- "asset_id"
- "custom1" (NCM Only)
- "custom2" (NCM Only)

> the "custom1" and "custom2" fields only exist in the NCM API in relation to the router_id and are not part of the device configuration.

### With all of this meta data about the router being included with your custom alert text, there are many integration and automation possibilities with webhooks!

## Creating an Application

### Use the File Menu or the Launcher to create a new Notebook for your app.

#### Or create a new folder for your notebooks and copy **csclient.py** into it.  

### Create your Application!

#### Start by importing the **csclient** module, then instantiating **CSClient()** and so forth!

#### Also, check out the [Sample Code included in this lab](/lab/tree/3.%20Sample%20Code)

### Happy Coding!

# Congratulations! 

## You have completed Part 3!

## Find Part 4 in the file browser to the left!